# Using a Temporal Convolutional Network for Daytrading
## Daniel Kalam, Sharvita Paithankar

In [55]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv1D, Dropout, Dense, Activation, BatchNormalization
import pandas as pd
from pandas_datareader import DataReader
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Gathering Data
Getting data for 100 stocks in the date range of April 2nd, 2018 to October 9th, 2020 from yahoo finance.

In [2]:
symbols = ['AAPL', 'TSLA', 'FB', 'GE', 'BRK', 'GOOGL', 'INTC', 'AMD', 'HPE', 'ZM',
          'CAKE', 'AET', 'F', 'KO', 'DDS', 'NVDA', 'NFLX', 'JPM', 'AMZN', 'MSFT']
#TODO: Add 80 more symbols.
source = 'yahoo'
start_date = pd.to_datetime('2018-02-04')
end_date = pd.to_datetime('2020-10-09')
stock_data = {}
for symbol in symbols:
    stock_data[symbol] = DataReader(symbol, source, start_date, end_date)

Create a data frame for each column in a stock's data frame.

In [3]:
open_data = {}
close_data = {}
high_data = {}
low_data = {}
volume_data = {}
adj_close_data = {}
for symbol in stock_data:
    open_data[symbol] = stock_data[symbol].Open
    close_data[symbol] = stock_data[symbol].Close
    high_data[symbol] = stock_data[symbol].High
    low_data[symbol] = stock_data[symbol].Low
    volume_data[symbol] = stock_data[symbol].Volume
    adj_close_data[symbol] = stock_data[symbol]['Adj Close']

## TensorFlow
### Converting the Data Into Tensors
Turn the data frames into tensorflow datatypes so that they can be processed by tensorflow.

In [93]:
open_train = tf.convert_to_tensor(np.array((10, 637, 1)))
for i in range(0, 10):
    key = list(open_data.keys())[i]
    #open_train[i] = tf.convert_to_tensor(np.array(open_data[key]).reshape(len(open_data[key]), 1))
open_aapl = open_data['AAPL']
open_aapl = open_aapl[0 : 637]
open_aapl = np.array(open_aapl).reshape(1, 637, 1)
open_aapl_train = tf.convert_to_tensor(open_aapl)
close_aapl = close_data['AAPL']
close_aapl = close_aapl[0 : 637]
close_aapl = np.array(close_aapl).reshape(1, 637, 1)

### Temporal Convolutional Network Class

In [103]:
filter_count = 20 # Amount of filters
filters = [] # Filter size for each residual block
kernel_size = 10 #Resolution of each filter
level = kernel_size
n = 0
while level <= 637:
    filters.append(filter_count)
    level+=kernel_size + (kernel_size-1)*2**n
    n+=1

In [104]:
class ResidualBlock(Layer):
    def __init__(self, filters, kernel_size, strides, dilation_rate, activation,
                trainable, dropout, dtype=None, activity_regularizer=None, **kwargs):
        super(ResidualBlock, self).__init__(trainable, dtype=dtype)
        self.activation = activation
        self.dropout = dropout
        self.filters = filters
        self.adjust_sample = None
        self.layer_norm = BatchNormalization(axis=-1)
        self.dilatedcausal1 = Conv1D(filters,
                                     kernel_size,
                                     strides,
                                     'causal',
                                     dilation_rate=dilation_rate)
        self.dilatedcausal2 = Conv1D(filters,
                                     kernel_size,
                                     strides,
                                     'causal',
                                     dilation_rate=dilation_rate)

    #Make the dropout based on the shape of the input
    def build(self, input_shape):
        self.drop1 = Dropout(self.dropout, input_shape)
        self.drop2 = Dropout(self.dropout, input_shape)
        if input_shape[2]!=filters:
            self.adjust_sample = Dense(self.filters)

    #The residual block processes the input
    def call(self, inputs, training):
        x = self.dilatedcausal1(inputs)
        x = self.layer_norm(x, training)
        x = self.activation(x)
        x = self.drop1(x, training) #If training is False, drop1 simply returns x
        x = self.dilatedcausal2(x)
        x = self.layer_norm(x, training)
        x = self.activation(x)
        x = self.drop2(x, training) #If training is False, drop2 simply returns x
        if self.adjust_sample is not None:
            inputs = self.adjust_sample(inputs)
        return self.activation(x+inputs)
        
class TCN(Model):
    def __init__(self, filters, kernel_size=2, dropout = 0.2, activation='relu',
                trainable=False, dtype=None, name=None,
                activity_regularizer=None, **kwargs):
        super(TCN, self).__init__()
        self.levels = []
        for i in range(0, len(filters)):
            self.levels.append(ResidualBlock(filters[i], kernel_size,
                                             1, 2**i, Activation(activation),
                                             trainable, dropout,
                                             dtype, activity_regularizer))
    
    #Running the input through each residual block
    def call(self, inputs, training=False):
        for r_block in self.levels:
            inputs = r_block(inputs, training)
        return inputs

In [105]:
tcn_model = TCN(filters, kernel_size, activation='sigmoid', trainable = True, dtype='float')
output = tcn_model(tf.cast(open_aapl_train, tf.float32))
print(output)

tf.Tensor(
[[[0.6073861  0.34409004 0.43042523 ... 0.49353698 0.39382455 0.8205135 ]
  [0.60738635 0.34409022 0.43042496 ... 0.49353704 0.39382467 0.8205135 ]
  [0.6073957  0.34409216 0.430414   ... 0.49353966 0.3938288  0.82051325]
  ...
  [0.586775   0.3139089  0.43800172 ... 0.4645725  0.41251302 0.83354807]
  [0.586775   0.31390893 0.43800166 ... 0.46457243 0.41251305 0.83354807]
  [0.58677495 0.31390882 0.43800163 ... 0.4645725  0.41251302 0.8335482 ]]], shape=(1, 637, 20), dtype=float32)
